<p style="text-align: center;">МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ РОССИЙСКОЙ ФЕДЕРАЦИИ
<p style="text-align: center;">Федеральное государственное автономное образовательное учреждение высшего образования «Самарский национальный исследовательский университет имени академика С. П. Королева» (Самарский университет)</p>
<br> <br> <br>
<p style="text-align: center;">Институт информатики и кибернетики
<p style="text-align: center;">Факультет информатики
<p style="text-align: center;">Кафедра программных систем
<br><br><br>
<p style="text-align: center;">ОТЧЁТ
<p style="text-align: center;">по лабораторной работе № 2
<p style="text-align: center;">«Введение в Spark с использованием Python (часть 1)»
<p style="text-align: center;">по курсу «Интеллектуальный анализ данных и большие данные»
<p style="text-align: center;">
<br><br><br><br><br><br><br><br>
<p style="text-align: right;">Выполнил: Яшин И.А.
<p style="text-align: right;">гр. 6132-020402D
<p style="text-align: right;">
<br><br><br><br><br><br><br><br><br>
<p style="text-align: center;">Самара 2025

In [ ]:
import os
from pyspark.sql import SparkSession

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
spark = SparkSession.builder.getOrCreate()

In [ ]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

In [ ]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

In [ ]:
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df

In [ ]:
# All DataFrames above result same.
df.show()
df.printSchema()

In [ ]:
df.show(1)

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

In [ ]:
df.show(1, vertical=True)

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [ ]:
df.select("a", "b", "c").describe().show()

In [ ]:
df.collect()

In [ ]:
df.take(1)

In [ ]:
df.toPandas()

In [ ]:
df.a

In [ ]:
from pyspark.sql import Column
from pyspark.sql.functions import upper

type(df.c) == type(upper(df.c)) == type(df.c.isNull())

In [ ]:
df.select(df.c).show()

In [ ]:
df.withColumn('upper_c', upper(df.c)).show()

In [ ]:
df.filter(df.a == 1).show()

In [ ]:
from pyspark.sql.pandas.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1

df.select(pandas_plus_one(df.a)).show()

In [ ]:
def pandas_filter_func(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.a == 1]

df.mapInPandas(pandas_filter_func, schema=df.schema).show()

In [ ]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

In [ ]:
df.groupby('color').avg().show()

In [ ]:
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 - pandas_df.v1.mean())

df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

In [ ]:
df1 = spark.createDataFrame(
    [(20000101, 1, 1.0), (20000101, 2, 2.0), (20000102, 1, 3.0), (20000102, 2, 4.0)],
    ('time', 'id', 'v1'))

df2 = spark.createDataFrame(
    [(20000101, 1, 'x'), (20000101, 2, 'y')],
    ('time', 'id', 'v2'))

def merge_ordered(l, r):
    return pd.merge_ordered(l, r)

df1.groupby('id').cogroup(df2.groupby('id')).applyInPandas(
    merge_ordered, schema='time int, id int, v1 double, v2 string').show()

In [ ]:
df.write.csv('foo.csv', header=True)
spark.read.csv('foo.csv', header=True).show()

In [ ]:
df.write.parquet('bar.parquet')
spark.read.parquet('bar.parquet').show()

In [ ]:
df.write.orc('zoo.orc')
spark.read.orc('zoo.orc').show()

In [ ]:
df.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

In [ ]:
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
    return s + 1

spark.udf.register("add_one", add_one)
spark.sql("SELECT add_one(v1) FROM tableA").show()

In [ ]:
from pyspark.sql.functions import expr

df.selectExpr('add_one(v1)').show()
df.select(expr('count(*)') > 0).show()